<a href="https://colab.research.google.com/github/HudaKhraiss/ADS2002-Group-Project/blob/main/ADS2002_Group_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/ranzcr-clip-catheter-line-classification/train.csv')

In [12]:
df.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [13]:
annotations = pd.read_csv('/content/drive/My Drive/ranzcr-clip-catheter-line-classification/train_annotations.csv')

In [14]:
annotations.head()

,StudyInstanceUID,label,data
0,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1487, 1279], [1477, 1168], [1472, 1052], [14..."
1,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1328, 7], [1347, 101], [1383, 193], [1400, 2..."
2,1.2.826.0.1.3680043.8.498.72921907356394389969...,CVC - Borderline,"[[801, 1207], [812, 1112], [823, 1023], [842, ..."
3,1.2.826.0.1.3680043.8.498.11697104485452001927...,CVC - Normal,"[[1366, 961], [1411, 861], [1453, 751], [1508,..."
4,1.2.826.0.1.3680043.8.498.87704688663091069148...,NGT - Normal,"[[1862, 14], [1845, 293], [1801, 869], [1716, ..."


In [15]:
df.isna().sum()

StudyInstanceUID              0
ETT - Abnormal                0
ETT - Borderline              0
ETT - Normal                  0
NGT - Abnormal                0
NGT - Borderline              0
NGT - Incompletely Imaged     0
NGT - Normal                  0
CVC - Abnormal                0
CVC - Borderline              0
CVC - Normal                  0
Swan Ganz Catheter Present    0
PatientID                     0
dtype: int64

In [16]:
df.duplicated().sum()

0

In [17]:
df.dtypes

StudyInstanceUID              object
ETT - Abnormal                 int64
ETT - Borderline               int64
ETT - Normal                   int64
NGT - Abnormal                 int64
NGT - Borderline               int64
NGT - Incompletely Imaged      int64
NGT - Normal                   int64
CVC - Abnormal                 int64
CVC - Borderline               int64
CVC - Normal                   int64
Swan Ganz Catheter Present     int64
PatientID                     object
dtype: object

In [18]:
df.shape

(30083, 13)

In [19]:
print("Number of unique patients: ", df["PatientID"].unique().shape[0])

Number of unique patients:  3255


In [20]:
idCounts = df['PatientID'].value_counts().reset_index()
idCounts.columns = ['PatientID', 'Number of Observations']
idCounts = idCounts.sort_values(by = 'Number of Observations', ascending = False)
idCounts.head()

,PatientID,Number of Observations
0,05029c63a,172
1,55073fece,167
2,26da0d5ad,148
3,8849382d0,130
4,34242119f,110


In [21]:
import plotly.express as px
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [22]:
fig = px.histogram(idCounts, x='Number of Observations', title='Distribution of Number of Observations per PatientIDs', template='plotly_white')
fig.show()

In [23]:
categories = ['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal','Swan Ganz Catheter Present']

categoryCounts = df[categories].sum(axis = 0).reset_index()
categoryCounts.columns = ['Malpositions', 'Number of Observations']

In [24]:
fig = px.bar(categoryCounts, x = 'Number of Observations', y = 'Malpositions', template = 'plotly_white', title = 'Line and tube positions')
fig.show()

In [25]:
countsDf = {'Position Type' : [], 'Malposition'  : [], 'Number of Observations' : []}
for Type in ['Normal','Abnormal','Borderline']:
    for malposition in ['ETT','NGT','CVC']:
        colName = f'{malposition} - {Type}'
        countsDf['Position Type'].append(Type)
        countsDf['Malposition'].append(malposition)
        val = df[colName].sum(axis = 0)
        countsDf['Number of Observations'].append(val)
countsDf = pd.DataFrame(countsDf)

In [26]:
fig = px.bar(countsDf, x = 'Number of Observations', y = 'Position Type', color = 'Malposition', barmode = 'stack', template = 'plotly_white')
fig.show()

In [39]:
df = df.drop(df[df["NGT - Incompletely Imaged"]==1].index)
df.shape

(27335, 13)

In [41]:
df = df.drop(df[df["Swan Ganz Catheter Present"] == 1].index)

0